### Q1. Getting the embeddings model

In [1]:
from sentence_transformers import SentenceTransformer
model_name = 'multi-qa-distilbert-cos-v1'
embedding_model = SentenceTransformer(model_name)

/ssd/ksu/pyenv/versions/3.12.2/envs/llm-zoomcamp/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
user_question = "I just discovered the course. Can I still join it?"

In [3]:
vector_user_question = embedding_model.encode(user_question)

In [4]:
vector_user_question[0]

0.07822263

In [5]:
dims = len(vector_user_question)

In [6]:
dims

768

In [7]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [8]:
len(documents)

948

In [9]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c02e79ef'}

In [10]:
course = "machine-learning-zoomcamp"
course_documents = [doc for doc in documents if doc["course"] == course]
len(course_documents)

375

### Q2. Creating the embeddings

In [11]:
embeddings = []
for doc in course_documents:
    qa_text = f'{doc['question']} {doc['text']}'
    qa_embedding = embedding_model.encode(qa_text)
    embeddings.append(qa_embedding)

In [12]:
import numpy as np

X = np.array(embeddings)
X.shape

(375, 768)

### Q3. Search

In [13]:
v = vector_user_question
scores = X.dot(v)

In [14]:
np.max(scores)

0.6506573

In [15]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=course_documents, embeddings=X)
vec_search_results = search_engine.search(v, num_results=5)

In [16]:
vec_search_results

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

### Q4. Hit-rate for our search engine

In [17]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [18]:
ground_truth[0]

{'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp',
 'document': '0227b872'}

In [19]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [21]:
from tqdm.auto import tqdm

relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    vector_question = embedding_model.encode(q['question'])
    results = search_engine.search(vector_question, num_results=5)
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

  0%|          | 0/1830 [00:00<?, ?it/s]

In [22]:
hit_rate(relevance_total)

0.9398907103825137

### Q5. Indexing with Elasticsearch

In [26]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

es_client.info()

ObjectApiResponse({'name': '29c3a5c13299', 'cluster_name': 'docker-cluster', 'cluster_uuid': '5cKUWYxkRO24GR54w_C6ag', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [27]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} ,
            "text_vector": {"type": "dense_vector", "dims": dims, "index": True, "similarity": "cosine"},
        }
    }
}

index_name = "machine-learning-zoomcamp-course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'machine-learning-zoomcamp-course-questions'})

In [29]:
docs_with_vectors = []
for i, doc in enumerate(course_documents):
    doc["text_vector"] = embeddings[i].tolist()
    docs_with_vectors.append(doc)

for doc in docs_with_vectors:
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

In [30]:
query = {
    "field": "text_vector",
    "query_vector": vector_user_question,
    "k": 5,
    "num_candidates": 10000, 
}

In [32]:
res = es_client.search(index=index_name, knn=query, source=["text", "section", "question", "course"])
res["hits"]["hits"][0]['_id']

'Au4nx5AB4aljRQSd2BBx'

### Q6. Hit-rate for Elasticsearch